In [2]:
# This information helps with debugging and getting support :)
import sys, platform
import pandas as pd
import bifacial_radiance as br
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("bifacial_radiance version ", br.__version__)

Working on a  Windows 10
Python version  3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]
Pandas version  1.5.3
bifacial_radiance version  0.4.4.dev25+g9486645.d20240925


# 21 - Weather to Module Performance
## Modeling Performance, an End to End Simulation

This tutorial shows how to use the new function on bifacial_radiance calculatePerformanceModule performance, as well as how to find CEC Module parameters.


In [3]:
import os
from pathlib import Path

testfolder = str(Path().resolve().parent.parent / 'bifacial_radiance' / 'TEMP' / 'Tutorial_21')

if not os.path.exists(testfolder): os.mkdir(testfolder)

# Another option using relative address; for some operative systems you might need '/' instead of '\'
# testfolder = os.path.abspath(r'..\..\bifacial_radiance\TEMP')  

print ("Your simulation will be stored in %s" % testfolder)


Your simulation will be stored in C:\Users\cdeline\Documents\Python Scripts\Bifacial_Radiance\bifacial_radiance\TEMP\Tutorial_21


In [4]:
import bifacial_radiance
import numpy as np
import pandas as pd
import pvlib 

bifacial_radiance.__version__

'0.4.4.dev25+g9486645.d20240925'

## Geting a CEC Module to pass into demo.makeModule

In [5]:
url = 'https://raw.githubusercontent.com/NREL/SAM/patch/deploy/libraries/CEC%20Modules.csv'
db = pd.read_csv(url, index_col=0) # Reading this might take 1 min or so, the database is big.

Find the module that you want. In this case we know it's a SunPower of model SPR-E19-310-COM. 

Make sure you select only 1 module from the database -- sometimes there are similar names.

In [6]:
modfilter2 = db.index.str.startswith('SunPower') & db.index.str.endswith('SPR-E19-310-COM')
print(modfilter2)
CECMod = db[modfilter2]
print(len(CECMod), " modules selected. Name of 1st entry: ", CECMod.index[0])

[False False False ... False False False]
1  modules selected. Name of 1st entry:  SunPower SPR-E19-310-COM


In [7]:
# Selecting only two times as examples
starttime = '01_13_11';  endtime = '01_13_12'
demo = bifacial_radiance.RadianceObj('tutorial_21', path = testfolder) # Create a RadianceObj 'object'
weatherfile = demo.getEPW(lat = 37.5, lon = -77.6)  # This location corresponds to Richmond, VA.
metdata = demo.readWeatherFile(weatherFile=weatherfile, starttime=starttime, endtime=endtime)
demo.setGround(0.2)

path = C:\Users\cdeline\Documents\Python Scripts\Bifacial_Radiance\bifacial_radiance\TEMP\Tutorial_21
Getting weather file: USA_VA_Richmond.724010_TMY2.epw
 ... OK!
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.


The CEC data should be passed into the ModuleObj, either at time of creation, or sometime before it is passed into makeScene.

In [8]:
mymodule = demo.makeModule(name='test-module', x=1, y=2, bifi=0.9, CECMod=CECMod) 


Module Name: test-module
Module test-module updated in module.json
Pre-existing .rad file objects\test-module.rad will be overwritten



The same data could instead be passed after the ModuleObj's definition, or at time of performance analysis:

In [9]:
mymodule.addCEC(CECMod)

In [10]:
# Let's make a second module, and set it to the default Prism Solar module type
mymodule2 = demo.makeModule(name='test', x=1, y=2, bifi=0.8, CECMod=None) 


Module Name: test
Module test updated in module.json
Pre-existing .rad file objects\test.rad will be overwritten



We're going to set up two scenes, each with a different module type!

In [11]:
sceneDict = {'tilt': 0, 'azimuth': 180, 'pitch': 5,'hub_height':1.5, 'nMods':5, 'nRows': 2}
trackerdict = demo.set1axis(metdata = metdata, cumulativesky = False)
trackerdict = demo.gendaylit1axis()
trackerdict = demo.makeScene1axis(trackerdict, module = mymodule, sceneDict = sceneDict)

Creating ~2 skyfiles. 
Created 2 skyfiles in /skies/

Making ~2 .rad files for gendaylit 1-axis workflow (this takes a minute..)
2 Radfiles created in /objects/


Make a second scene with the other module type

In [12]:
sceneDict2 = {'tilt': 0, 'azimuth': 180, 'pitch': 5,'hub_height':2.5, 'nMods':2, 'nRows': 1, 'originx': -15}
trackerdict = demo.makeScene1axis(trackerdict, module = mymodule2, sceneDict=sceneDict2, append=True)



Making ~2 .rad files for gendaylit 1-axis workflow (this takes a minute..)
2 Radfiles created in /objects/


In [13]:
# Compile both scenes into one octfile.  Run 2 different analyses, one on each scene
trackerdict = demo.makeOct1axis()
trackerdict = demo.analysis1axis(sensorsy=3, append=False)
trackerdict = demo.analysis1axis(sensorsy=3, sceneNum=1)


Making 2 octfiles in root directory.
Created 1axis_2021-01-13_1100.oct
Created 1axis_2021-01-13_1200.oct

Analysis1axis: append=False. Over-writing any existing `AnalysisObj` in trackerdict.

Linescan in process: 1axis_2021-01-13_1100_Scene0_Row1_Module3_Front
Linescan in process: 1axis_2021-01-13_1100_Scene0_Row1_Module3_Back
Saved: results\irr_1axis_2021-01-13_1100_Scene0_Row1_Module3.csv
Index: 2021-01-13_1100. Wm2Front: 254.37783333333334. Wm2Back: 40.0465
Linescan in process: 1axis_2021-01-13_1200_Scene0_Row1_Module3_Front
Linescan in process: 1axis_2021-01-13_1200_Scene0_Row1_Module3_Back
Saved: results\irr_1axis_2021-01-13_1200_Scene0_Row1_Module3.csv
Index: 2021-01-13_1200. Wm2Front: 253.61329999999998. Wm2Back: 37.96939666666667
Linescan in process: 1axis_2021-01-13_1100_Scene1_Row1_Module1_Front
Linescan in process: 1axis_2021-01-13_1100_Scene1_Row1_Module1_Back
Saved: results\irr_1axis_2021-01-13_1100_Scene1_Row1_Module1.csv
Index: 2021-01-13_1100. Wm2Front: 253.51493333333

In [14]:
# Include an AgriPV groundscan too
trackerdict = demo.analysis1axisground(sceneNum=1, sensorsground=10, customname='Silvanas_')

Linescan in process: 1axis_groundscan_2021-01-13_1100Silvanas__Row1_Module1_Front
Saved: results\irr_1axis_groundscan_2021-01-13_1100Silvanas__Row1_Module1.csv
Index: 2021-01-13_1100. Wm2Ground: 204.88986. sensorsground: 10
Linescan in process: 1axis_groundscan_2021-01-13_1200Silvanas__Row1_Module1_Front
Saved: results\irr_1axis_groundscan_2021-01-13_1200Silvanas__Row1_Module1.csv
Index: 2021-01-13_1200. Wm2Ground: 237.75890999999996. sensorsground: 10


In [15]:
# show the initial irradiance results before continuing:
demo.results

,timestamp,name,modNum,rowNum,sceneNum,x,y,z,rearZ,mattype,rearMat,Wm2Front,Wm2Back,backRatio,rearX,rearY,surf_azm,surf_tilt,theta
0,2021-01-13_1100,1axis_2021-01-13_1100_Scene0,3,1,0,"[0.3734448, 0.01462469, -0.3441954]","[2.28669e-17, 8.955042e-19, -2.107589e-17]","[1.166863, 1.51507, 1.863277]","[1.151075, 1.499282, 1.847489]","[a2.0.a0.test-module.6457, a2.0.a0.test-module...","[a2.0.a0.test-module.2310, a2.0.a0.test-module...","[253.9854, 254.9895, 254.1586]","[39.46907, 39.66438, 41.00605]","[0.1553983599121843, 0.15555238332408464, 0.16...","[0.3581237, -0.000696414, -0.3595166]","[2.192875e-17, -4.264306e-20, -2.201404e-17]",90.0,44.14,-44.14
1,2021-01-13_1100,1axis_2021-01-13_1100_Scene1,1,1,1,"[-14.62656, -14.98538, -15.3442]","[2.28669e-17, 8.955042e-19, -2.107589e-17]","[2.166863, 2.51507, 2.863277]","[2.151075, 2.499282, 2.847489]","[a0.0.a0.test.6457, a0.0.a0.test.6457, a0.0.a0...","[a0.0.a0.test.2310, a0.0.a0.test.2310, a0.0.a0...","[253.0249, 253.8044, 253.7155]","[52.58993, 53.09413, 53.07791]","[0.20784405865170325, 0.20919227880888272, 0.2...","[-14.64188, -15.0007, -15.35952]","[2.192875e-17, -4.264306e-20, -2.201404e-17]",90.0,44.14,-44.14
2,2021-01-13_1100,1axis_groundscan_2021-01-13_1100Silvanas_,1,1,1,"[-15.0, -14.44444, -13.88889, -13.33333, -12.7...","[0.0, 3.401797e-17, 6.803593e-17, 1.020539e-16...","[0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.0...",NaN,"[groundplane, groundplane, groundplane, ground...",NaN,"[199.43860000000004, 201.74889999999996, 204.7...",NaN,NaN,NaN,NaN,90.0,44.14,-44.14
3,2021-01-13_1200,1axis_2021-01-13_1200_Scene0,3,1,0,"[0.473756, 0.007594116, -0.4585678]","[2.900919e-17, 4.650055e-19, -2.807918e-17]","[1.338767, 1.519579, 1.700391]","[1.318255, 1.499068, 1.67988]","[a2.0.a0.test-module.6457, a2.0.a0.test-module...","[a2.0.a0.test-module.2310, a2.0.a0.test-module...","[253.5078, 253.6133, 253.7188]","[37.62749, 37.65656, 38.62414]","[0.14842676072783273, 0.14847964014647438, 0.1...","[0.4658003, -0.0003616246, -0.4665235]","[2.852204e-17, -2.214312e-20, -2.856633e-17]",90.0,21.20,-21.20
4,2021-01-13_1200,1axis_2021-01-13_1200_Scene1,1,1,1,"[-14.52624, -14.99241, -15.45857]","[2.900919e-17, 4.650055e-19, -2.807918e-17]","[2.338767, 2.519579, 2.700391]","[2.318255, 2.499068, 2.67988]","[a0.0.a0.test.6457, a0.0.a0.test.6457, a0.0.a0...","[a0.0.a0.test.2310, a0.0.a0.test.2310, a0.0.a0...","[252.92009999999996, 252.98069999999998, 253.0...","[50.52337, 50.86739, 50.98965999999999]","[0.19975941113651652, 0.20107142137158537, 0.2...","[-14.5342, -15.00036, -15.46652]","[2.852204e-17, -2.214312e-20, -2.856633e-17]",90.0,21.20,-21.20
5,2021-01-13_1200,1axis_groundscan_2021-01-13_1200Silvanas_,1,1,1,"[-15.0, -14.44444, -13.88889, -13.33333, -12.7...","[0.0, 3.401797e-17, 6.803593e-17, 1.020539e-16...","[0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.0...",NaN,"[groundplane, groundplane, groundplane, ground...",NaN,"[227.8342, 230.13659999999996, 233.87209999999...",NaN,NaN,NaN,NaN,90.0,21.20,-21.20


## Calculating the Performance and Exporting the Results to a CSV

In [16]:
# Calculate performance.
pd.set_option('display.max_columns', 1000); 
compiledResults = demo.calculatePerformance1axis()
print(f'\nCompiled results:\n')
display(compiledResults)


No CECModule data passed; using default for Prism Solar BHC72-400

Compiled results:



,timestamp,name,modNum,rowNum,sceneNum,x,y,z,rearZ,mattype,rearMat,Wm2Front,Wm2Back,backRatio,rearX,rearY,POA_eff,Gfront_mean,Grear_mean,Pout_raw,Pout_Gfront,BGG,BGE,Mismatch,Pout,Wind Speed,DNI,DHI,GHI,module_CEC_name
0,2021-01-13_1100,1axis_2021-01-13_1100_Scene0,3,1,0,"[0.3734448, 0.01462469, -0.3441954]","[2.28669e-17, 8.955042e-19, -2.107589e-17]","[1.166863, 1.51507, 1.863277]","[1.151075, 1.499282, 1.847489]","[a2.0.a0.test-module.6457, a2.0.a0.test-module...","[a2.0.a0.test-module.2310, a2.0.a0.test-module...","[253.9854, 254.9895, 254.1586]","[39.46907, 39.66438, 41.00605]","[0.1553983599121843, 0.15555238332408464, 0.16...","[0.3581237, -0.000696414, -0.3595166]","[2.192875e-17, -4.264306e-20, -2.201404e-17]",290.419683,254.377833,40.046500,91.831552,80.346050,14.168628,14.295042,0.031494,91.802630,3.8,144,149,211,SunPower SPR-E19-310-COM
1,2021-01-13_1100,1axis_2021-01-13_1100_Scene1,1,1,1,"[-14.62656, -14.98538, -15.3442]","[2.28669e-17, 8.955042e-19, -2.107589e-17]","[2.166863, 2.51507, 2.863277]","[2.151075, 2.499282, 2.847489]","[a0.0.a0.test.6457, a0.0.a0.test.6457, a0.0.a0...","[a0.0.a0.test.2310, a0.0.a0.test.2310, a0.0.a0...","[253.0249, 253.8044, 253.7155]","[52.58993, 53.09413, 53.07791]","[0.20784405865170325, 0.20919227880888272, 0.2...","[-14.64188, -15.0007, -15.35952]","[2.192875e-17, -4.264306e-20, -2.201404e-17]",295.851459,253.514933,52.920657,123.731803,106.057563,16.699815,16.664762,0.008722,123.721012,3.8,144,149,211,Prism Solar Technologies Inc. BHC72-400
2,2021-01-13_1100,1axis_groundscan_2021-01-13_1100Silvanas_,1,1,1,"[-15.0, -14.44444, -13.88889, -13.33333, -12.7...","[0.0, 3.401797e-17, 6.803593e-17, 1.020539e-16...","[0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.0...",NaN,"[groundplane, groundplane, groundplane, ground...",NaN,"[199.43860000000004, 201.74889999999996, 204.7...",NaN,NaN,NaN,NaN,204.889860,204.889860,0.000000,85.625380,85.625380,0.000000,0.000000,0.000000,85.625380,3.8,144,149,211,Prism Solar Technologies Inc. BHC72-400
3,2021-01-13_1200,1axis_2021-01-13_1200_Scene0,3,1,0,"[0.473756, 0.007594116, -0.4585678]","[2.900919e-17, 4.650055e-19, -2.807918e-17]","[1.338767, 1.519579, 1.700391]","[1.318255, 1.499068, 1.67988]","[a2.0.a0.test-module.6457, a2.0.a0.test-module...","[a2.0.a0.test-module.2310, a2.0.a0.test-module...","[253.5078, 253.6133, 253.7188]","[37.62749, 37.65656, 38.62414]","[0.14842676072783273, 0.14847964014647438, 0.1...","[0.4658003, -0.0003616246, -0.4665235]","[2.852204e-17, -2.214312e-20, -2.856633e-17]",287.785757,253.613300,37.969397,90.325515,79.509085,13.474237,13.604017,0.020285,90.307193,3.9,97,200,249,SunPower SPR-E19-310-COM
4,2021-01-13_1200,1axis_2021-01-13_1200_Scene1,1,1,1,"[-14.52624, -14.99241, -15.45857]","[2.900919e-17, 4.650055e-19, -2.807918e-17]","[2.338767, 2.519579, 2.700391]","[2.318255, 2.499068, 2.67988]","[a0.0.a0.test.6457, a0.0.a0.test.6457, a0.0.a0...","[a0.0.a0.test.2310, a0.0.a0.test.2310, a0.0.a0...","[252.92009999999996, 252.98069999999998, 253.0...","[50.52337, 50.86739, 50.98965999999999]","[0.19975941113651652, 0.20107142137158537, 0.2...","[-14.5342, -15.00036, -15.46652]","[2.852204e-17, -2.214312e-20, -2.856633e-17]",293.615512,252.980733,50.793473,121.980576,105.120321,16.062401,16.039006,0.008120,121.970671,3.9,97,200,249,Prism Solar Technologies Inc. BHC72-400
5,2021-01-13_1200,1axis_groundscan_2021-01-13_1200Silvanas_,1,1,1,"[-15.0, -14.44444, -13.88889, -13.33333, -12.7...","[0.0, 3.401797e-17, 6.803593e-17, 1.020539e-16...","[0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.0...",NaN,"[groundplane, groundplane, groundplane, ground...",NaN,"[227.8342, 230.13659999999996, 233.87209999999...",NaN,NaN,NaN,NaN,237.758910,237.758910,0.000000,98.778151,98.778151,0.000000,0.000000,0.000000,98.778151,3.9,97,200,249,Prism Solar Technologies Inc. BHC72-400


In [16]:
demo.exportTrackerDict(savefile=os.path.join('results','Final_Results.csv'),reindex=False)
pd.read_csv(os.path.join('results','Final_Results.csv'))

Exporting TrackerDict
3
3
1
1


,Unnamed: 0,timestamp,rowNum,modNum,sceneNum,name,Wm2Front,Wm2Back,DNI,DHI,GHI,temp_air,wind_speed,theta,surf_tilt,surf_azm,POA_eff,Gfront_mean,Grear_mean,Pout_raw,Mismatch,Pout,Pout_Gfront
0,0,2021-01-13_1100,1,3,0,1axis_2021-01-13_1100_Scene0,[254.0681 254.9558 254.3216],[39.50778 39.82523 40.90701],144,149,211,4.6,3.8,-44.14,44.14,90.0,290.520506,254.448500,40.080007,91.863627,0.028544,91.837406,80.368605
1,1,2021-01-13_1100,1,1,1,1axis_2021-01-13_1100_Scene1,[253.0643 253.8386 253.637 ],[52.60152 53.10697 52.92946],144,149,211,4.6,3.8,-44.14,44.14,90.0,295.816753,253.513300,52.879317,123.717364,0.008745,123.706545,106.056879
2,2,2021-01-13_1100,1,1,1,1axis_groundscan_2021-01-13_1100Silvanas_,[198.7301 201.027 204.6268 206.4026 207.4362 ...,NaN,144,149,211,4.6,3.8,-44.14,44.14,90.0,205.031480,205.031480,0.000000,85.685065,0.000000,85.685065,85.685065
3,3,2021-01-13_1200,1,3,0,1axis_2021-01-13_1200_Scene0,[252.768 252.9163 253.3978],[37.82638 37.77132 38.61288],97,200,249,6.5,3.9,-21.20,21.20,90.0,287.290541,253.027367,38.070193,90.169005,0.017078,90.153606,79.323353
4,4,2021-01-13_1200,1,1,1,1axis_2021-01-13_1200_Scene1,[252.9312 253.0649 253.1985],[50.65721 51.0674 50.72511],97,200,249,6.5,3.9,-21.20,21.20,90.0,293.718125,253.064867,50.816573,122.023014,0.007130,122.014314,105.155337
5,5,2021-01-13_1200,1,1,1,1axis_groundscan_2021-01-13_1200Silvanas_,[228.0597 229.9865 233.2411 237.3016 239.1257 ...,NaN,97,200,249,6.5,3.9,-21.20,21.20,90.0,237.717610,237.717610,0.000000,98.760926,0.000000,98.760926,98.760926


## Now look at gencumulativesky tracking workflow

In [17]:
starttime = '01_13_11';  endtime = '12_13_12'
demo = bifacial_radiance.RadianceObj('tutorial_21', path = testfolder) # Create a RadianceObj 'object'
weatherfile = demo.getEPW(lat = 37.5, lon = -77.6)  # This location corresponds to Richmond, VA.
metdata = demo.readWeatherFile(weatherFile=weatherfile, starttime=starttime, endtime=endtime)
#metdata = demo.readWeatherFile(weatherFile=weatherfile)
demo.setGround(0.2)
mymodule = demo.makeModule(name='test-module', x=1, y=2, bifi=0.9, CECMod=CECMod) 

path = C:\Users\cdeline\Documents\Python Scripts\Bifacial_Radiance\bifacial_radiance\TEMP\Tutorial_21
Getting weather file: USA_VA_Richmond.724010_TMY2.epw
 ... OK!
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.

Module Name: test-module
Module test-module updated in module.json
Pre-existing .rad file objects\test-module.rad will be overwritten



In [18]:
sceneDict = {'tilt': 0, 'azimuth': 180, 'pitch': 5,'hub_height':1.5, 'nMods':5, 'nRows': 2}
trackerdict = demo.set1axis(metdata=metdata, cumulativesky=True, limit_angle=15, backtrack=False)
trackerdict = demo.genCumSky1axis()
trackerdict = demo.makeScene1axis(trackerdict, module = mymodule, sceneDict = sceneDict)
trackerdict = demo.makeOct1axis()
trackerdict = demo.analysis1axis(modWanted = [2,4], sensorsy=3)
trackerdict = demo.analysis1axisground(sensorsground=10)

Saving file EPWs\1axis_-15.0.csv, # points: 1875
Saving file EPWs\1axis_-10.0.csv, # points: 241
Saving file EPWs\1axis_0.0.csv, # points: 2
Saving file EPWs\1axis_5.0.csv, # points: 242
Saving file EPWs\1axis_10.0.csv, # points: 33
Saving file EPWs\1axis_15.0.csv, # points: 1683
message: There were 1833 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_-15.0.rad
message: There were 238 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_-10.0.rad
message: There were 2 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_0.0.rad
message: There were 239 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_5.0.rad
message: There were 33 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_10.0.rad
message: There were 1661 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_15.0.rad

Makin

In [19]:
results = demo.calculatePerformance1axis() # saves to demo.compiledResults and results/Cumulative_Results.csv
print('\nCompiled results:\n')
display(demo.compiledResults)


Compiled results:



,rowNum,modNum,sceneNum,BGG,Gfront_mean,Grear_mean,POA_eff,Wm2Front,Wm2Back
0,1,2,0,11.338927,1.655285e+06,208546.127300,"[1845372.5030866666, 1840463.504476667, 184309...","[1658556.0944, 1658093.2835, 1649204.3125, nan...","[211208.8218, 205754.3789, 208675.18120000002]"
1,1,3,0,NaN,1.125704e+06,NaN,"[0.0, 0.0, 0.0]","[633171.4857999999, 711829.5524, 862250.9423, ...","[nan, nan, nan]"
2,1,4,0,10.334268,1.658672e+06,190457.407267,"[1832873.54007, 1826657.0977800002, 1830721.81...","[1665464.9309999999, 1665355.3404000003, 16451...","[193556.7293, 186649.57120000003, 191165.92130..."


In [20]:
# Results are also automatically saved in \results\Cumulative_Results.csv
pd.read_csv(os.path.join('results','Cumulative_Results.csv'))

,rowNum,modNum,sceneNum,BGG,Gfront_mean,Grear_mean,POA_eff,Wm2Front,Wm2Back
0,1,2,0,11.339,1655284.563,208546.127,"[1845372.5030866666, 1840463.504476667, 184309...","[1658556.0944, 1658093.2835, 1649204.3125, nan...","[211208.8218, 205754.3789, 208675.18120000002]"
1,1,3,0,NaN,1125703.831,NaN,"[0.0, 0.0, 0.0]","[633171.4857999999, 711829.5524, 862250.9423, ...","[nan, nan, nan]"
2,1,4,0,10.334,1658672.484,190457.407,"[1832873.54007, 1826657.0977800002, 1830721.81...","[1665464.9309999999, 1665355.3404000003, 16451...","[193556.7293, 186649.57120000003, 191165.92130..."
